# Code for Andrew Cho's COS IW Project (Movie Recommendations)

The following notebook implements:

## Recommendation Models:
1. Collaborative Filtering (CF) - Matrix factorization using SVD
2. Content-Based Filtering (CBF) - Semantic embeddings from movie text
3. Hybrid - switching combination (CF for warm items, CBF fallback for cold items)

## Evaluation Regimes:
- Warm: Standard per-user 80/20 temporal split on dataset
- Cold Item: True item cold-start (items appearing for first time after temporal cutoff)
- Few-shot Users: Users with limited history (k=1,3,5 ratings in training)

## Metrics:
- Regression: MAE, RMSE
- Ranking: Recall@K, NDCG@K, Precision@K (with negative sampling)


AI Usage: AI code generation was utilized, specifically for cache and dir path creation and implementation, evaluation framework testing, code modularization, function comments, clear print statements.

## 1. Setup & Imports

1. Clone the repo.
2. The github repo comes with a requirements.txt file. Create a virtual enviorment with those package versions installed then run this notebook using that virtual enviorment.
3. Set up a folder called data in the same directory as this notebook and add the files from this link: 
4. Run!

In [ ]:
# libraries
import os
import random
import pickle
import warnings
from pathlib import Path
from datetime import datetime
from collections import defaultdict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

from sklearn.linear_model import Ridge
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize

from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split as surprise_split

from sentence_transformers import SentenceTransformer

import torch

warnings.filterwarnings('ignore')

# variable configuration
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# paths to store intermediate results like figures, embeddings, etc.
DATA_DIR = Path('data')
OUTPUT_DIR = Path('outputs')
TABLES_DIR = OUTPUT_DIR / 'tables'
FIGURES_DIR = OUTPUT_DIR / 'figures'
CACHE_DIR = OUTPUT_DIR / 'cache'

# create output directories
for d in [TABLES_DIR, FIGURES_DIR, CACHE_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# experiment parameters
K = 10  # Top-K for ranking metrics (Precision@10, Recall@10, NDCG@10)
N_NEG = 100  # Number of negative samples per positive (for ranking evaluation)
MAX_USERS_RANK = 5000  # Max users for ranking evaluation (samples from test set for efficiency)
COLD_CUTOFF_QUANTILE = 0.8  # Temporal cutoff for cold-start split (~Oct 2018)
RELEVANCE_THRESHOLD = 4.0  # Minimum rating to consider "relevant" (4.0 = "liked it")
                           # items with rating ≥4.0 count as "hits" in ranking metrics

# hybrid alpha values to sweep (including 1.0 for pure CF)
ALPHA_VALUES = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]


## 2. Data Loading & Validation


In [ ]:
# load ratings data
ratings = pd.read_csv(DATA_DIR / 'ratings.csv')
print(f"Number of ratings: {len(ratings):,}")

# load movies data
movies = pd.read_pickle(DATA_DIR / 'movies_enriched_with_semantic.pkl')
print(f"Number of movies: {len(movies):,}")

# select relevant columns - use semantic_text for CBF
movies = movies[['movieId', 'semantic_text']].copy()
movies.columns = ['movieId', 'text']

movies.head()

In [ ]:
# check for duplicate (userId, movieId) pairs
n_before = len(ratings)
ratings = ratings.drop_duplicates(subset=['userId', 'movieId'], keep='first')
n_after = len(ratings)
n_duplicates = n_before - n_after

print(f"duplicate pairs removed: {n_duplicates:,}")
if n_duplicates > 0:
    print(f"  before: {n_before:,}, after: {n_after:,}")

# basic stats
n_ratings = len(ratings)
n_users = ratings['userId'].nunique()
n_items = ratings['movieId'].nunique()
rating_min = ratings['rating'].min()
rating_max = ratings['rating'].max()

ts_min = ratings['timestamp'].min()
ts_max = ratings['timestamp'].max()
date_min = datetime.fromtimestamp(ts_min)
date_max = datetime.fromtimestamp(ts_max)

print(f"\nratings statistics:")
print(f"total ratings: {n_ratings:,}")
print(f"unique users: {n_users:,}")
print(f"unique items: {n_items:,}")
print(f"rating range: [{rating_min}, {rating_max}]")
print(f"date range: {date_min.strftime('%Y-%m-%d')} to {date_max.strftime('%Y-%m-%d')}")

# sparsity
total_possible = n_users * n_items
sparsity = 1 - (n_ratings / total_possible)
print(f"sparsity: {sparsity*100:.4f}%")

# distribution
ratings_per_user = ratings.groupby('userId').size()
ratings_per_item = ratings.groupby('movieId').size()

print(f"ratings per user: mean={ratings_per_user.mean():.2f}, median={ratings_per_user.median():.0f}")
print(f"ratings per item: mean={ratings_per_item.mean():.2f}, median={ratings_per_item.median():.0f}")

# check overlap
rated_items = set(ratings['movieId'].unique())
movie_items = set(movies['movieId'].unique())
overlap = rated_items.intersection(movie_items)
missing_in_movies = rated_items - movie_items

print(f"\nmovies data check:")
print(f"movies in file: {len(movie_items):,}")
print(f"items with ratings: {len(rated_items):,}")
print(f"overlap: {len(overlap):,}")
print(f"items missing movie text: {len(missing_in_movies):,}")

# filter to items with movie text
if len(missing_in_movies) > 0:
    print(f"\nfiltering to items with movie text")
    ratings = ratings[ratings['movieId'].isin(movie_items)]
    print(f"  ratings after filter: {len(ratings):,}")
    n_users = ratings['userId'].nunique()
    n_items = ratings['movieId'].nunique()
    print(f"  users: {n_users:,}, items: {n_items:,}")

# dtypes
ratings['userId'] = ratings['userId'].astype(np.int32)
ratings['movieId'] = ratings['movieId'].astype(np.int32)
ratings['rating'] = ratings['rating'].astype(np.float32)
ratings['timestamp'] = ratings['timestamp'].astype(np.int64)

print(f"\ndata types verified")


## 3. Helper Utilities


In [ ]:
# HELPER UTILITIES

def build_user_item_map(df: pd.DataFrame) -> dict:
    """Build user -> set of items mapping from a DataFrame."""
    user_items = defaultdict(set)
    for uid, mid in zip(df['userId'].values, df['movieId'].values):
        user_items[uid].add(mid)
    return dict(user_items)


def build_item_user_map(df: pd.DataFrame) -> dict:
    """Build item -> set of users mapping from a DataFrame."""
    item_users = defaultdict(set)
    for uid, mid in zip(df['userId'].values, df['movieId'].values):
        item_users[mid].add(uid)
    return dict(item_users)


def sample_negatives(candidate_pool: set, forbidden: set, n: int, rng: np.random.Generator) -> list:
    """
    Sample n negatives from candidate_pool, excluding forbidden items.
    
    Args:
        candidate_pool: set of all possible items to sample from
        forbidden: set of items to exclude (user's train/test items)
        n: number of negatives to sample
        rng: numpy random generator for reproducibility
        
    Returns:
        List of sampled negative item IDs
    """
    available = list(candidate_pool - forbidden)
    if len(available) < n:
        return available  # Return all available if not enough
    return list(rng.choice(available, size=n, replace=False))


def create_id_mappings(items: np.ndarray):
    """Create bidirectional ID mappings for items."""
    unique_items = np.unique(items)
    item_to_idx = {item: idx for idx, item in enumerate(unique_items)}
    idx_to_item = {idx: item for item, idx in item_to_idx.items()}
    return item_to_idx, idx_to_item


def print_split_stats(train_df: pd.DataFrame, test_df: pd.DataFrame, name: str):
    """Print statistics for a train/test split."""
    print(f"\n{name} Split Statistics:")
    print(f"  Train: {len(train_df):,} ratings, {train_df['userId'].nunique():,} users, {train_df['movieId'].nunique():,} items")
    print(f"  Test:  {len(test_df):,} ratings, {test_df['userId'].nunique():,} users, {test_df['movieId'].nunique():,} items")
    
    # check overlap
    train_users = set(train_df['userId'].unique())
    test_users = set(test_df['userId'].unique())
    train_items = set(train_df['movieId'].unique())
    test_items = set(test_df['movieId'].unique())
    
    print(f"  Test users in train: {len(test_users.intersection(train_users)):,} / {len(test_users):,}")
    print(f"  Test items in train: {len(test_items.intersection(train_items)):,} / {len(test_items):,}")


print("Helper utilities defined.")


## 4. Split Builders (3 Evaluation Regimes)

### 4.1 Warm Split (Per-user temporal 80/20)


In [ ]:
# 4.1 WARM SPLIT: Per-user temporal 80/20

def build_warm_split(df: pd.DataFrame, train_ratio: float = 0.8):
    """
    Create warm split: per-user temporal split.
    First train_ratio% of each user's interactions -> train
    Remaining -> test
    """
    # sort by user and timestamp
    df_sorted = df.sort_values(['userId', 'timestamp']).reset_index(drop=True)
    
    # compute cumulative count per user
    df_sorted['user_cum_count'] = df_sorted.groupby('userId').cumcount() + 1
    df_sorted['user_total'] = df_sorted.groupby('userId')['userId'].transform('count')
    df_sorted['user_position'] = df_sorted['user_cum_count'] / df_sorted['user_total']
    
    # split based on position
    train_mask = df_sorted['user_position'] <= train_ratio
    
    train_df = df_sorted[train_mask][['userId', 'movieId', 'rating', 'timestamp']].copy()
    test_df = df_sorted[~train_mask][['userId', 'movieId', 'rating', 'timestamp']].copy()
    
    return train_df, test_df

# build warm split
print("building WARM split...")
train_warm, test_warm = build_warm_split(ratings, train_ratio=0.8)

# print statistics
print_split_stats(train_warm, test_warm, "WARM")

# leakage check: verify timestamps don't leak
# for warm split, each user's test timestamps should be >= train timestamps
print("\nLeakage Check (per-user temporal ordering):")
train_max_ts = train_warm.groupby('userId')['timestamp'].max()
test_min_ts = test_warm.groupby('userId')['timestamp'].min()
common_users = train_max_ts.index.intersection(test_min_ts.index)
violations = (train_max_ts[common_users] > test_min_ts[common_users]).sum()
print(f"  Users with timestamp violations: {violations} / {len(common_users)}")
assert violations == 0, "Timestamp leakage detected in warm split!"
print("  ✓ No timestamp leakage detected")


### 4.2 True Item Cold-Start Split (Global temporal cutoff)


In [ ]:
# 4.2 TRUE ITEM COLD-START SPLIT

def build_cold_item_split(df: pd.DataFrame, q_cut: float = 0.8):
    """
    Create true item cold-start split.
    
    1. Global temporal cutoff at q_cut quantile
    2. Cold items = items that appear for the FIRST TIME after cutoff
    3. test_cold = interactions with cold items after cutoff
    
    Args:
        df: ratings DataFrame
        q_cut: quantile for temporal cutoff (default 0.8 = 80th percentile)
        
    Returns:
        train_cold: ratings before cutoff
        test_cold: ratings after cutoff for cold items only
        cold_items: set of cold item IDs
        T: timestamp cutoff value
    """
    # compute cutoff timestamp
    T = df['timestamp'].quantile(q_cut)
    T_date = datetime.fromtimestamp(T)
    print(f"Temporal cutoff T: {T} ({T_date.strftime('%Y-%m-%d')})")
    
    # split by cutoff
    train_cold = df[df['timestamp'] <= T].copy()
    test_after = df[df['timestamp'] > T].copy()
    
    print(f"Ratings before cutoff (train_cold): {len(train_cold):,}")
    print(f"Ratings after cutoff: {len(test_after):,}")
    
    # find first appearance of each item (over ALL data)
    first_time_per_item = df.groupby('movieId')['timestamp'].min()
    
    # cold items: first appearance is AFTER cutoff T
    cold_items = set(first_time_per_item[first_time_per_item > T].index)
    print(f"cold items (first appear after T): {len(cold_items):,}")
    
    # test_cold: only cold items from test_after
    test_cold = test_after[test_after['movieId'].isin(cold_items)].copy()
    print(f"cold item interactions in test: {len(test_cold):,}")
    print(f"Users in test_cold: {test_cold['userId'].nunique():,}")
    
    return train_cold, test_cold, cold_items, T


# build cold item split
print("building COLD ITEM split...")
train_cold, test_cold, cold_items, T_cutoff = build_cold_item_split(ratings, q_cut=COLD_CUTOFF_QUANTILE)

# CRITICAL LEAKAGE CHECK
train_cold_items = set(train_cold['movieId'].unique())
leakage = cold_items.intersection(train_cold_items)

print(f"\n{'='*60}")
print("COLD ITEM LEAKAGE CHECK")
print(f"{'='*60}")
print(f"cold items: {len(cold_items):,}")
print(f"Train items: {len(train_cold_items):,}")
print(f"Intersection (SHOULD BE ZERO): {len(leakage)}")

assert len(leakage) == 0, f"LEAKAGE DETECTED! {len(leakage)} cold items found in training set!"
print("✓ No leakage: cold items are truly unseen in training")

print_split_stats(train_cold, test_cold, "COLD ITEM")


### 4.3 Few-shot User Splits (k=1, 3, 5)


In [ ]:
# 4.3 FEW-SHOT USER SPLITS

def build_fewshot_splits(df: pd.DataFrame, k_values: list = [1, 3, 5]):
    """
    Create few-shot user splits for different k values.
    
    For each k:
    - train_k = first k interactions per user (sorted by timestamp)
    - test_k = remaining interactions
    - Only include users with > k interactions
    
    Args:
        df: ratings DataFrame
        k_values: list of k values to create splits for
        
    Returns:
        Dictionary: {k: (train_k, test_k)}
    """
    # sort by user and timestamp once
    df_sorted = df.sort_values(['userId', 'timestamp']).reset_index(drop=True)
    
    # add position within each user
    df_sorted['user_position'] = df_sorted.groupby('userId').cumcount() + 1
    user_counts = df_sorted.groupby('userId').size()
    
    fewshot_splits = {}
    
    for k in k_values:
        print(f"\nBuilding few-shot split for k={k}...")
        
        # filter to users with > k interactions
        valid_users = user_counts[user_counts > k].index
        df_valid = df_sorted[df_sorted['userId'].isin(valid_users)]
        
        # split: first k -> train, rest -> test
        train_k = df_valid[df_valid['user_position'] <= k][['userId', 'movieId', 'rating', 'timestamp']].copy()
        test_k = df_valid[df_valid['user_position'] > k][['userId', 'movieId', 'rating', 'timestamp']].copy()
        
        fewshot_splits[k] = (train_k, test_k)
        
        print(f"  Users with > {k} interactions: {len(valid_users):,}")
        print_split_stats(train_k, test_k, f"Few-shot k={k}")
    
    return fewshot_splits


# build few-shot splits
print("building FEW-SHOT USER splits...")
fewshot_splits = build_fewshot_splits(ratings, k_values=[1, 3, 5])

print(f"\n{'='*60}")
print("FEW-SHOT SPLITS SUMMARY")
print(f"{'='*60}")
for k, (train_k, test_k) in fewshot_splits.items():
    print(f"k={k}: Train={len(train_k):,}, Test={len(test_k):,}")


## 5. Model Implementations

### 5.1 Collaborative Filtering (CF) - Surprise SVD

We use the Surprise library's SVD implementation which includes:
- Biased matrix factorization (global mean, user bias, item bias)
- Regularization
- Optimized C++ backend for efficiency


In [ ]:
# collaborative filtering using surprise SVD

class CFModel:
    # matrix factorization with user/item biases
    
    def __init__(self, n_factors=100, n_epochs=20, lr_all=0.005, reg_all=0.02, random_state=SEED):
        self.n_factors = n_factors
        self.n_epochs = n_epochs
        self.lr_all = lr_all
        self.reg_all = reg_all
        self.random_state = random_state
        self.model = None
        self.train_items_set = None
        self.train_users_set = None
        self.global_mean = None
        self.user_means = None
        self.reader = None
        
    def fit(self, train_df: pd.DataFrame, verbose=True):
        if verbose:
            print(f"fitting cf model (svd) with {self.n_factors} factors, {self.n_epochs} epochs")
        
        # store training set info
        self.train_items_set = set(train_df['movieId'].unique())
        self.train_users_set = set(train_df['userId'].unique())
        self.global_mean = train_df['rating'].mean()
        self.user_means = train_df.groupby('userId')['rating'].mean().to_dict()
        
        # convert to surprise format
        rating_min = train_df['rating'].min()
        rating_max = train_df['rating'].max()
        self.reader = Reader(rating_scale=(rating_min, rating_max))
        
        data = Dataset.load_from_df(
            train_df[['userId', 'movieId', 'rating']], 
            self.reader
        )
        trainset = data.build_full_trainset()
        
        # train svd
        self.model = SVD(
            n_factors=self.n_factors,
            n_epochs=self.n_epochs,
            lr_all=self.lr_all,
            reg_all=self.reg_all,
            random_state=self.random_state,
            verbose=verbose
        )
        self.model.fit(trainset)
        
        if verbose:
            print(f"cf model fitted - items: {len(self.train_items_set):,}, users: {len(self.train_users_set):,}")
        
        return self
    
    def predict(self, pairs_df: pd.DataFrame) -> np.ndarray:
        # predict ratings using svd
        predictions = []
        for uid, mid in zip(pairs_df['userId'].values, pairs_df['movieId'].values):
            pred = self.model.predict(uid, mid)
            predictions.append(pred.est)
        return np.array(predictions)
    
    def predict_fallback(self, pairs_df: pd.DataFrame) -> np.ndarray:
        # fallback: use user mean or global mean for cold items
        predictions = []
        for uid in pairs_df['userId'].values:
            if uid in self.user_means:
                predictions.append(self.user_means[uid])
            else:
                predictions.append(self.global_mean)
        return np.array(predictions)
    
    def save(self, path: Path):
        """Save model to disk."""
        with open(path, 'wb') as f:
            pickle.dump({
                'model': self.model,
                'train_items_set': self.train_items_set,
                'train_users_set': self.train_users_set,
                'global_mean': self.global_mean,
                'user_means': self.user_means,
                'reader': self.reader,
                'n_factors': self.n_factors,
                'n_epochs': self.n_epochs
            }, f)
        print(f"CF model saved to {path}")
    
    @classmethod
    def load(cls, path: Path):
        """Load model from disk."""
        with open(path, 'rb') as f:
            data = pickle.load(f)
        
        model = cls(n_factors=data['n_factors'], n_epochs=data['n_epochs'])
        model.model = data['model']
        model.train_items_set = data['train_items_set']
        model.train_users_set = data['train_users_set']
        model.global_mean = data['global_mean']
        model.user_means = data['user_means']
        model.reader = data['reader']
        print(f"CF model loaded from {path}")
        return model


print("cfmodel defined")


### 5.2 Content-Based Filtering (CBF) - Semantic Embeddings


In [ ]:
# 5.2.1 EMBEDDING COMPUTATION & CACHING

def compute_or_load_embeddings(movies_df: pd.DataFrame, model_name: str, cache_dir: Path) -> tuple:
    """
    Compute or load cached embeddings for movie text.
    
    Args:
        movies_df: DataFrame with columns [movieId, text]
        model_name: Name of the sentence-transformer model
        cache_dir: Directory for caching embeddings
        
    Returns:
        embeddings: numpy array of shape (n_movies, embedding_dim)
        movieId_to_idx: dict mapping movieId -> embedding row index
        idx_to_movieId: dict mapping embedding row index -> movieId
    """
    # create safe filename for cache
    safe_name = model_name.replace('/', '_')
    emb_path = cache_dir / f'{safe_name}_emb.npy'
    mapping_path = cache_dir / f'{safe_name}_mapping.pkl'
    
    if emb_path.exists() and mapping_path.exists():
        print(f"loading cached embeddings for {model_name}...")
        embeddings = np.load(emb_path)
        with open(mapping_path, 'rb') as f:
            mappings = pickle.load(f)
        movieId_to_idx = mappings['movieId_to_idx']
        idx_to_movieId = mappings['idx_to_movieId']
        print(f"  Loaded embeddings: {embeddings.shape}")
        return embeddings, movieId_to_idx, idx_to_movieId
    
    print(f"Computing embeddings for {model_name}...")
    
    # load sentence transformer model
    st_model = SentenceTransformer(model_name)
    
    # sort movies by movieId for consistent ordering
    movies_sorted = movies_df.sort_values('movieId').reset_index(drop=True)
    
    # create mappings
    movieId_to_idx = {mid: idx for idx, mid in enumerate(movies_sorted['movieId'].values)}
    idx_to_movieId = {idx: mid for mid, idx in movieId_to_idx.items()}
    
    # get texts
    texts = movies_sorted['text'].fillna('').tolist()
    
    # compute embeddings in batches
    print(f"  Encoding {len(texts):,} movie texts...")
    embeddings = st_model.encode(
        texts, 
        batch_size=128, 
        show_progress_bar=True,
        convert_to_numpy=True
    )
    
    # normalize embeddings for cosine similarity
    embeddings = normalize(embeddings, axis=1)
    
    # cache
    print(f"  Caching embeddings to {emb_path}...")
    np.save(emb_path, embeddings)
    with open(mapping_path, 'wb') as f:
        pickle.dump({'movieId_to_idx': movieId_to_idx, 'idx_to_movieId': idx_to_movieId}, f)
    
    print(f"  Embeddings computed and cached: {embeddings.shape}")
    return embeddings, movieId_to_idx, idx_to_movieId


print("Embedding computation function defined.")


In [ ]:
# content-based filtering using semantic embeddings

class CBFModel:
    # ranking model: computes similarity between user profiles and items
    # user profiles are weighted average of item embeddings
    
    def __init__(self, use_rating_weights=True):
        self.use_rating_weights = use_rating_weights
        self.user_profiles = None
        self.item_embeddings = None
        self.movieId_to_idx = None
        self.idx_to_movieId = None
        self.train_users = None
        
    def fit(self, train_df: pd.DataFrame, item_embeddings: np.ndarray, 
            movieId_to_idx: dict, verbose=True):
        if verbose:
            print("fitting cbf model")
        
        self.item_embeddings = item_embeddings
        self.movieId_to_idx = movieId_to_idx
        self.idx_to_movieId = {v: k for k, v in movieId_to_idx.items()}
        
        # compute user means
        self.global_mean = train_df['rating'].mean()
        self.user_means = train_df.groupby('userId')['rating'].mean().to_dict()
        self.train_users = set(train_df['userId'].unique())
        
        # build user profiles
        if verbose:
            print("  Building user profiles...")
        
        user_items_ratings = train_df.groupby('userId').apply(
            lambda x: list(zip(x['movieId'].values, x['rating'].values))
        ).to_dict()
        
        self.user_profiles = {}
        emb_dim = item_embeddings.shape[1]
        
        for uid, items_ratings in tqdm(user_items_ratings.items(), 
                                       disable=not verbose, 
                                       desc="  User profiles"):
            profile = np.zeros(emb_dim)
            total_weight = 0
            
            user_mean = self.user_means.get(uid, self.global_mean)
            
            for mid, rating in items_ratings:
                if mid not in self.movieId_to_idx:
                    continue
                    
                idx = self.movieId_to_idx[mid]
                
                if self.use_rating_weights:
                    # weight by deviation from user mean (directional preference)
                    weight = rating - user_mean + 3.0  # Add offset to keep positive
                else:
                    weight = 1.0
                
                profile += weight * item_embeddings[idx]
                total_weight += abs(weight)
            
            if total_weight > 0:
                profile /= total_weight
                # normalize for cosine similarity
                norm = np.linalg.norm(profile)
                if norm > 0:
                    profile /= norm
            
            self.user_profiles[uid] = profile
        
        if verbose:
            print(f"cbf model fitted - {len(self.user_profiles):,} user profiles created")
        
        return self
    
    def predict(self, pairs_df: pd.DataFrame) -> np.ndarray:
        # compute similarity scores (not ratings)
        scores = []
        
        for uid, mid in zip(pairs_df['userId'].values, pairs_df['movieId'].values):
            if uid not in self.user_profiles or mid not in self.movieId_to_idx:
                scores.append(0.0)  # unseen users/items get 0 similarity
                continue
            
            user_profile = self.user_profiles[uid]
            item_emb = self.item_embeddings[self.movieId_to_idx[mid]]
            
            # cosine similarity (dot product of normalized vectors)
            sim = np.dot(user_profile, item_emb)
            scores.append(sim)
        
        return np.array(scores)
    
    def save(self, path: Path):
        with open(path, 'wb') as f:
            pickle.dump({
                'user_profiles': self.user_profiles,
                'item_embeddings': self.item_embeddings,
                'movieId_to_idx': self.movieId_to_idx,
                'idx_to_movieId': self.idx_to_movieId,
                'train_users': self.train_users,
                'global_mean': self.global_mean,
                'user_means': self.user_means,
                'use_rating_weights': self.use_rating_weights
            }, f)
        print(f"cbf model saved to {path}")
    
    @classmethod
    def load(cls, path: Path):
        with open(path, 'rb') as f:
            data = pickle.load(f)
        
        model = cls(use_rating_weights=data['use_rating_weights'])
        model.user_profiles = data['user_profiles']
        model.item_embeddings = data['item_embeddings']
        model.movieId_to_idx = data['movieId_to_idx']
        model.idx_to_movieId = data['idx_to_movieId']
        model.train_users = data['train_users']
        model.global_mean = data['global_mean']
        model.user_means = data['user_means']
        
        print(f"cbf model loaded from {path} ({len(model.user_profiles):,} profiles)")
        return model


print("cbfmodel defined")


In [ ]:
# 5.2.3 EMBEDDING MODEL COMPARISON

def compare_embedding_models(train_df: pd.DataFrame, val_df: pd.DataFrame, 
                            movies_df: pd.DataFrame, model_names: list,
                            cache_dir: Path, n_sample: int = 10000):
    """
    Compare embedding models by evaluating CBF performance on validation set.
    
    Args:
        train_df: Training ratings
        val_df: Validation ratings (subset of test for quick eval)
        movies_df: Movies with text
        model_names: List of embedding model names to compare
        cache_dir: Directory for caching
        n_sample: Number of validation samples to use
        
    Returns:
        results_df: DataFrame with MAE/RMSE for each model
        best_model_name: Name of best performing model
    """
    print(f"\n{'='*60}")
    print("EMBEDDING MODEL COMPARISON")
    print(f"{'='*60}")
    
    # sample validation set
    if len(val_df) > n_sample:
        val_sample = val_df.sample(n=n_sample, random_state=SEED)
    else:
        val_sample = val_df
    
    print(f"Validation sample size: {len(val_sample):,}")
    
    results = []
    
    for model_name in model_names:
        print(f"\n--- {model_name}")
        
        # load or compute embeddings
        embeddings, movieId_to_idx, _ = compute_or_load_embeddings(
            movies_df, model_name, cache_dir
        )
        
        # train CBF model
        cbf = CBFModel(use_rating_weights=True)
        cbf.fit(train_df, embeddings, movieId_to_idx, verbose=False)
        
        # predict on validation
        preds = cbf.predict(val_sample)
        true_ratings = val_sample['rating'].values
        
        # compute metrics
        mae = np.mean(np.abs(preds - true_ratings))
        rmse = np.sqrt(np.mean((preds - true_ratings) ** 2))
        
        results.append({
            'model': model_name,
            'MAE': mae,
            'RMSE': rmse,
            'embedding_dim': embeddings.shape[1]
        })
        
        print(f"  MAE: {mae:.4f}, RMSE: {rmse:.4f}, Dim: {embeddings.shape[1]}")
    
    results_df = pd.DataFrame(results)
    
    # find best model (by MAE)
    best_idx = results_df['MAE'].idxmin()
    best_model_name = results_df.loc[best_idx, 'model']
    
    print(f"\n{'='*60}")
    print(f"BEST MODEL: {best_model_name} (MAE: {results_df.loc[best_idx, 'MAE']:.4f})")
    print(f"{'='*60}")
    
    return results_df, best_model_name


print("Embedding comparison function defined.")


### 5.3 Hybrid Model (switching: CF for warm items, CBF fallback for cold)


In [ ]:
# hybrid model: switching strategy for cold-start items

class HybridModel:
    # uses cf for warm items, cbf for cold items
    # no blending since cf returns ratings and cbf returns similarity
    
    def __init__(self, cf_model: CFModel, cbf_model: CBFModel):
        self.cf_model = cf_model
        self.cbf_model = cbf_model
        
    def predict(self, pairs_df: pd.DataFrame) -> np.ndarray:
        """
        Predict scores using switching strategy.
        
        Returns:
        - For warm items: CF ratings (0.5-5.0) - proven best for regression
        - For cold items: CBF similarity (0-1) - only model that works
        
        Note: Returns mixed units, but this is appropriate since:
        - Warm items: CF has learned item factors → use ratings
        - Cold items: CF cannot predict → must use CBF similarity
        
        Args:
            pairs_df: DataFrame with columns [userId, movieId]
            
        Returns:
            Array of predictions (mixed units: ratings for warm, similarity for cold)
        """
        # get predictions from both models
        cf_preds = self.cf_model.predict(pairs_df)  # Ratings (0.5-5.0)
        cbf_preds = self.cbf_model.predict(pairs_df)  # Similarity (0-1)
        
        # determine which items are warm (in CF training set)
        warm_mask = pairs_df['movieId'].isin(self.cf_model.train_items_set).values
        
        # switching: Switch between CF and CBF based on item warmth
        # NO blending - use the right model for each item
        predictions = np.where(
            warm_mask,
            cf_preds,   # Warm: CF predictions (ratings)
            cbf_preds   # Cold: CBF predictions (similarity)
        )
        
        return predictions
    
    def predict_for_cold_regime(self, pairs_df: pd.DataFrame) -> np.ndarray:
        """
        For cold regime evaluation: use CBF for cold items, CF fallback otherwise.
        This is specific to the cold-start evaluation where all items are cold.
        """
        return self.cbf_model.predict(pairs_df)


def tune_hybrid_alpha(cf_model: CFModel, cbf_model: CBFModel, 
                      val_df: pd.DataFrame, alpha_values: list) -> tuple:
    """
    Tune hybrid alpha parameter on validation set.
    
    Args:
        cf_model: Trained CF model
        cbf_model: Trained CBF model
        val_df: Validation DataFrame
        alpha_values: List of alpha values to try
        
    Returns:
        best_alpha: Best alpha value
        results_df: DataFrame with metrics for each alpha
    """
    print(f"\n{'='*60}")
    print("HYBRID ALPHA TUNING")
    print(f"{'='*60}")
    print(f"Validation size: {len(val_df):,}")
    
    results = []
    true_ratings = val_df['rating'].values
    
    # pre-compute base predictions
    cf_preds = cf_model.predict(val_df)
    cbf_preds = cbf_model.predict(val_df)
    warm_mask = val_df['movieId'].isin(cf_model.train_items_set).values
    
    for alpha in alpha_values:
        # blend predictions
        hybrid_preds = np.where(
            warm_mask,
            alpha * cf_preds + (1 - alpha) * cbf_preds,
            cbf_preds
        )
        
        mae = np.mean(np.abs(hybrid_preds - true_ratings))
        rmse = np.sqrt(np.mean((hybrid_preds - true_ratings) ** 2))
        
        results.append({
            'alpha': alpha,
            'MAE': mae,
            'RMSE': rmse
        })
        
        print(f"  alpha={alpha:.2f}: MAE={mae:.4f}, RMSE={rmse:.4f}")
    
    results_df = pd.DataFrame(results)
    
    # find best alpha
    best_idx = results_df['MAE'].idxmin()
    best_alpha = results_df.loc[best_idx, 'alpha']
    
    print(f"\nBest alpha: {best_alpha} (MAE: {results_df.loc[best_idx, 'MAE']:.4f})")
    
    return best_alpha, results_df


print("HybridModel class and alpha tuning function defined.")


## 6. Evaluation Metrics


In [ ]:
# 6.1 REGRESSION METRICS

def compute_regression_metrics(y_true: np.ndarray, y_pred: np.ndarray) -> dict:
    """
    Compute regression metrics: MAE and RMSE.
    
    Args:
        y_true: True ratings
        y_pred: Predicted ratings
        
    Returns:
        Dictionary with MAE and RMSE
    """
    mae = np.mean(np.abs(y_true - y_pred))
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
    return {'MAE': mae, 'RMSE': rmse}


def evaluate_regression(predict_fn, test_df: pd.DataFrame) -> dict:
    """
    Evaluate model on regression metrics.
    
    Args:
        predict_fn: Function that takes pairs_df and returns predictions
        test_df: Test DataFrame with [userId, movieId, rating]
        
    Returns:
        Dictionary with metrics
    """
    predictions = predict_fn(test_df)
    true_ratings = test_df['rating'].values
    return compute_regression_metrics(true_ratings, predictions)


print("Regression metrics defined.")


In [ ]:
# ranking metrics with negative sampling

def precision_at_k(ranked_items: list, positives: set, k: int) -> float:
    # proportion of top-k that are relevant
    if k == 0:
        return 0.0
    top_k = set(ranked_items[:k])
    return len(top_k.intersection(positives)) / k


def recall_at_k(ranked_items: list, positives: set, k: int) -> float:
    # proportion of positives found in top-k
    if len(positives) == 0:
        return 0.0
    top_k = set(ranked_items[:k])
    return len(top_k.intersection(positives)) / len(positives)


def dcg_at_k(ranked_items: list, positives: set, k: int) -> float:
    # discounted cumulative gain (binary relevance)
    dcg = 0.0
    for i, item in enumerate(ranked_items[:k]):
        if item in positives:
            dcg += 1.0 / np.log2(i + 2)
    return dcg


def ndcg_at_k(ranked_items: list, positives: set, k: int) -> float:
    # normalized dcg
    dcg = dcg_at_k(ranked_items, positives, k)
    
    # ideal dcg: all positives at the top
    ideal_n = min(len(positives), k)
    idcg = sum(1.0 / np.log2(i + 2) for i in range(ideal_n))
    
    if idcg == 0:
        return 0.0
    return dcg / idcg


def evaluate_ranking(predict_fn, train_df: pd.DataFrame, test_df: pd.DataFrame,
                     candidate_pool: set, K: int = 10, N_neg: int = 100,
                     max_users: int = 5000, seed: int = SEED, 
                     relevance_threshold: float = 4.0) -> dict:
    """
    Evaluate ranking metrics with negative sampling.
    
    Protocol per user:
    1. positives = items in test rated >= relevance_threshold
    2. forbidden = train items + ALL test items (avoid false negatives)
    3. negatives = sample N_neg from candidate_pool excluding forbidden
    4. candidates = positives + negatives (shuffled to avoid ordering bias)
    5. Score all candidates, rank by score (with tie-breaking noise)
    6. Compute Recall@K, NDCG@K, Precision@K
    
    Note: Candidates are shuffled and tiny noise is added to scores to ensure
    fair ranking when scores are equal (e.g., CF_fallback gives same score to all items)
    
    Args:
        predict_fn: Function that takes pairs_df and returns scores
        train_df: Training DataFrame
        test_df: Test DataFrame
        candidate_pool: Set of item IDs to sample negatives from
        K: Top-K for metrics
        N_neg: Number of negatives to sample per user
        max_users: Maximum users to evaluate (for efficiency)
        seed: Random seed
        relevance_threshold: Minimum rating to consider an item "relevant" (default 4.0)
        
    Returns:
        Dictionary with average metrics
    """
    rng = np.random.default_rng(seed)
    
    # build user -> train items mapping
    train_user_items = build_user_item_map(train_df)
    
    # build user -> ALL test items (for forbidden set to avoid false negatives)
    all_test_user_items = build_user_item_map(test_df)
    
    # build user -> RELEVANT test items (rating >= threshold) for metrics
    test_relevant = test_df[test_df['rating'] >= relevance_threshold]
    relevant_user_items = build_user_item_map(test_relevant)
    
    print(f"  Relevance threshold: rating >= {relevance_threshold}")
    print(f"  Relevant test items: {len(test_relevant):,} / {len(test_df):,} ({100*len(test_relevant)/len(test_df):.1f}%)")
    
    # get users who have at least one relevant item
    test_users = list(relevant_user_items.keys())
    
    # sample users if too many
    if len(test_users) > max_users:
        print(f"  Sampling {max_users:,} users from {len(test_users):,} for ranking eval")
        test_users = list(rng.choice(test_users, size=max_users, replace=False))
    
    # metrics accumulators
    all_precision = []
    all_recall = []
    all_ndcg = []
    
    for uid in tqdm(test_users, desc="  Ranking eval"):
        # positives = items in test rated >= threshold (for metrics)
        positives = relevant_user_items.get(uid, set())
        if len(positives) == 0:
            continue
        
        # forbidden items: train + ALL test items (not just relevant ones)
        # this prevents sampling any test item as a negative
        train_items = train_user_items.get(uid, set())
        all_test_items = all_test_user_items.get(uid, set())
        forbidden = train_items.union(all_test_items)
        
        # sample negatives
        negatives = sample_negatives(candidate_pool, forbidden, N_neg, rng)
        if len(negatives) == 0:
            continue
        
        # candidates = positives + negatives
        # IMPORTANT: Shuffle candidates to avoid ordering bias when scores are equal
        # (e.g., CF_fallback gives same score to all items, so original order would bias results)
        candidates = list(positives) + negatives
        shuffle_indices = rng.permutation(len(candidates))
        candidates_shuffled = [candidates[i] for i in shuffle_indices]
        
        # create pairs DataFrame for scoring
        pairs_df = pd.DataFrame({
            'userId': [uid] * len(candidates_shuffled),
            'movieId': candidates_shuffled
        })
        
        # get scores
        scores = predict_fn(pairs_df)
        
        # rank by score (descending) with random tie-breaking
        # add tiny random noise to break ties fairly when scores are equal
        noise = rng.uniform(0, 1e-10, size=len(scores))
        scores_with_noise = scores + noise
        ranked_indices = np.argsort(-scores_with_noise)
        ranked_items = [candidates_shuffled[i] for i in ranked_indices]
        
        # compute metrics
        all_precision.append(precision_at_k(ranked_items, positives, K))
        all_recall.append(recall_at_k(ranked_items, positives, K))
        all_ndcg.append(ndcg_at_k(ranked_items, positives, K))
    
    n_evaluated = len(all_precision)
    
    return {
        f'Precision@{K}': np.mean(all_precision) if all_precision else 0.0,
        f'Recall@{K}': np.mean(all_recall) if all_recall else 0.0,
        f'NDCG@{K}': np.mean(all_ndcg) if all_ndcg else 0.0,
        'n_users_evaluated': n_evaluated
    }


print("Ranking metrics with negative sampling defined.")


## 7. Run Experiments and Save Results

### 7.1 Warm Regime Experiments


In [ ]:
# 7.1.1 LOAD EMBEDDINGS (all-MiniLM-L6-v2)

print()
print("STEP 1: LOAD EMBEDDINGS")
print()

# use all-MiniLM-L6-v2 embeddings (already computed/cached)
best_embedding_model = 'all-MiniLM-L6-v2'
print(f"Using embedding model: {best_embedding_model}")

# load embeddings
best_embeddings, best_movieId_to_idx, best_idx_to_movieId = compute_or_load_embeddings(
    movies, best_embedding_model, CACHE_DIR
)

print(f"Embeddings loaded: {best_embeddings.shape}")


In [ ]:
# 7.1.2 TRAIN CF MODEL (WARM REGIME)

print("\n" + "="*70)
print("STEP 2: TRAIN CF MODEL (Surprise SVD)")
print()

# check for cached model
cf_warm_cache_path = CACHE_DIR / 'cf_model_warm.pkl'

if cf_warm_cache_path.exists():
    print("loading cached CF model...")
    cf_warm = CFModel.load(cf_warm_cache_path)
else:
    cf_warm = CFModel(n_factors=100, n_epochs=20, random_state=SEED)
    cf_warm.fit(train_warm, verbose=True)
    cf_warm.save(cf_warm_cache_path)


In [ ]:
# 7.1.3 TRAIN CBF MODEL WITH BEST EMBEDDING (WARM REGIME)

print("\n" + "="*70)
print(f"STEP 3: TRAIN CBF MODEL with {best_embedding_model}")
print()

# load best embeddings
best_embeddings, best_movieId_to_idx, best_idx_to_movieId = compute_or_load_embeddings(
    movies, best_embedding_model, CACHE_DIR
)

# train or load CBF model with caching
cbf_warm_cache_path = CACHE_DIR / f"cbf_warm_{best_embedding_model.replace('/', '_')}.pkl"

if cbf_warm_cache_path.exists():
    print(f"loading cached CBF model from {cbf_warm_cache_path}...")
    cbf_warm = CBFModel.load(cbf_warm_cache_path)
else:
    print("training new CBF model...")
    cbf_warm = CBFModel(use_rating_weights=True)
    cbf_warm.fit(train_warm, best_embeddings, best_movieId_to_idx, verbose=True)
    cbf_warm.save(cbf_warm_cache_path)


In [ ]:
# 7.1.4 CREATE switching HYBRID MODEL

print("\n" + "="*70)
print("STEP 4: CREATE switching HYBRID MODEL")
print()

# note: No alpha tuning needed for switching hybrid!
# the hybrid uses a simple gating strategy:
# - Warm items → CF (only model that works)
# - Cold items → CBF (only model that works)
# no blending of incompatible units (ratings vs similarity)

print("creating switching hybrid model (CF for warm items, CBF for cold items)...")
hybrid_warm = HybridModel(cf_warm, cbf_warm)
print("switching hybrid model created")


In [ ]:
# 7.1.5 FULL WARM REGIME EVALUATION

print("\n" + "="*70)
print("STEP 5: FULL WARM REGIME EVALUATION")
print()

# WHAT WE'RE TESTING: Standard recommendation (both users and items are warm)
# - All users have training history
# - All test items exist in training set (CF has learned factors)
# - This is the "easiest" regime - CF should perform best
# - CBF: Ranks by similarity (no rating prediction, so MAE/RMSE = NaN)
# - Hybrid: Uses CF for all items (since all are warm), should match CF

# candidate pool for warm regime: items in training set
warm_candidate_pool = set(train_warm['movieId'].unique())
print(f"warm candidate pool size: {len(warm_candidate_pool):,}")

warm_results = []

# CF: Rating prediction model
print(f"\n--- Evaluating CF")
reg_metrics_cf = evaluate_regression(cf_warm.predict, test_warm)
print(f"  Regression: MAE={reg_metrics_cf['MAE']:.4f}, RMSE={reg_metrics_cf['RMSE']:.4f}")

print(f"  Computing ranking metrics...")
rank_metrics_cf = evaluate_ranking(
    cf_warm.predict, train_warm, test_warm,
    warm_candidate_pool, K=K, N_neg=N_NEG, max_users=MAX_USERS_RANK,
    relevance_threshold=RELEVANCE_THRESHOLD
)
print(f"  Ranking: Precision@{K}={rank_metrics_cf[f'Precision@{K}']:.4f}, "
      f"Recall@{K}={rank_metrics_cf[f'Recall@{K}']:.4f}, "
      f"NDCG@{K}={rank_metrics_cf[f'NDCG@{K}']:.4f}")

warm_results.append({
    'Model': 'CF',
    'Regime': 'Warm',
    **reg_metrics_cf,
    **{k: v for k, v in rank_metrics_cf.items() if k != 'n_users_evaluated'}
})

# CBF: Ranking-only model (similarity scores)
print(f"\n--- Evaluating CBF")
print(f"  (Ranking-only: using similarity scores)")

print(f"  Computing ranking metrics...")
rank_metrics_cbf = evaluate_ranking(
    cbf_warm.predict, train_warm, test_warm,
    warm_candidate_pool, K=K, N_neg=N_NEG, max_users=MAX_USERS_RANK,
    relevance_threshold=RELEVANCE_THRESHOLD
)
print(f"  Ranking: Precision@{K}={rank_metrics_cbf[f'Precision@{K}']:.4f}, "
      f"Recall@{K}={rank_metrics_cbf[f'Recall@{K}']:.4f}, "
      f"NDCG@{K}={rank_metrics_cbf[f'NDCG@{K}']:.4f}")
print(f"  Regression: N/A (CBF is a ranking model, not a rating predictor)")

warm_results.append({
    'Model': 'CBF',
    'Regime': 'Warm',
    'MAE': float('nan'),
    'RMSE': float('nan'),
    **{k: v for k, v in rank_metrics_cbf.items() if k != 'n_users_evaluated'}
})

# hybrid: Blends CF and CBF
print(f"\n--- Evaluating Hybrid")
reg_metrics_hybrid = evaluate_regression(hybrid_warm.predict, test_warm)
print(f"  Regression: MAE={reg_metrics_hybrid['MAE']:.4f}, RMSE={reg_metrics_hybrid['RMSE']:.4f}")

print(f"  Computing ranking metrics...")
rank_metrics_hybrid = evaluate_ranking(
    hybrid_warm.predict, train_warm, test_warm,
    warm_candidate_pool, K=K, N_neg=N_NEG, max_users=MAX_USERS_RANK,
    relevance_threshold=RELEVANCE_THRESHOLD
)
print(f"  Ranking: Precision@{K}={rank_metrics_hybrid[f'Precision@{K}']:.4f}, "
      f"Recall@{K}={rank_metrics_hybrid[f'Recall@{K}']:.4f}, "
      f"NDCG@{K}={rank_metrics_hybrid[f'NDCG@{K}']:.4f}")

warm_results.append({
    'Model': 'Hybrid',
    'Regime': 'Warm',
    **reg_metrics_hybrid,
    **{k: v for k, v in rank_metrics_hybrid.items() if k != 'n_users_evaluated'}
})

# create results DataFrame
warm_results_df = pd.DataFrame(warm_results)
warm_results_df.to_csv(TABLES_DIR / 'warm_metrics.csv', index=False)

print(f"\n{'='*60}")
print("WARM REGIME RESULTS")
print(f"{'='*60}")
print(warm_results_df.to_string(index=False))
print(f"\nResults saved to {TABLES_DIR / 'warm_metrics.csv'}")


### 7.2 Cold Item Regime Experiments

**Note:** For cold items, CF item-specific predictions are undefined. We report CF_fallback (user mean / global mean) as a baseline.


In [ ]:
# cold item regime experiments

print("\ncold item regime experiments")

# filter cold items to only those with embeddings
print("\nfiltering cold items to those with embeddings")
cold_items_with_embeddings = cold_items.intersection(set(best_movieId_to_idx.keys()))
missing_embeddings = cold_items - cold_items_with_embeddings

print(f"original cold items: {len(cold_items):,}")
print(f"cold items with embeddings: {len(cold_items_with_embeddings):,}")
print(f"missing embeddings: {len(missing_embeddings):,}")

if len(missing_embeddings) > 0:
    print(f"\nwarning: {len(missing_embeddings)} cold items missing embeddings")
    print(f"filtering test_cold to ensure cbf can make predictions")
    
    # filter test_cold to only include items with embeddings
    test_cold = test_cold[test_cold['movieId'].isin(cold_items_with_embeddings)].copy()
    cold_items = cold_items_with_embeddings
    
    print(f"Filtered test_cold: {len(test_cold):,} ratings remaining")

print("\ntraining models on train_cold")

# train cf on train_cold
cf_cold_cache_path = CACHE_DIR / 'cf_model_cold.pkl'

if cf_cold_cache_path.exists():
    print("loading cached cf model (cold)")
    cf_cold = CFModel.load(cf_cold_cache_path)
else:
    cf_cold = CFModel(n_factors=100, n_epochs=20, random_state=SEED)
    cf_cold.fit(train_cold, verbose=True)
    cf_cold.save(cf_cold_cache_path)

# train or load cbf on train_cold
cbf_cold_cache_path = CACHE_DIR / f"cbf_cold_{best_embedding_model.replace('/', '_')}.pkl"

if cbf_cold_cache_path.exists():
    print(f"loading cached cbf model from {cbf_cold_cache_path}")
    cbf_cold = CBFModel.load(cbf_cold_cache_path)
else:
    print("training new cbf model for cold regime")
    cbf_cold = CBFModel(use_rating_weights=True)
    cbf_cold.fit(train_cold, best_embeddings, best_movieId_to_idx, verbose=True)
    cbf_cold.save(cbf_cold_cache_path)

# create switching hybrid for cold regime
hybrid_cold = HybridModel(cf_cold, cbf_cold)

print(f"\n{'='*60}")
print("IMPORTANT NOTE ON COLD ITEM EVALUATION")
print(f"{'='*60}")
print("CF item-specific predictions are UNDEFINED for cold items.")
print("We report CF_fallback (user mean / global mean) as a baseline.")
print("For the Hybrid model on cold items, only CBF predictions are used (switching).")
print(f"{'='*60}")


In [ ]:
# cold item evaluation

print("\nevaluating on test_cold (cold items only)")

# candidate pool: cold items only
cold_candidate_pool = cold_items
print(f"cold candidate pool size: {len(cold_candidate_pool):,}")
print(f"test cold size: {len(test_cold):,}")

cold_results = []

# evaluate models on cold split

print("\nevaluating models on cold items")

# cbf is a ranking model for cold-start
# uses cosine similarity scores (not ratings)
# evaluate only on ranking metrics (precision@k, recall@k, ndcg)
# regression metrics (mae/rmse) not applicable for cbf on cold-start

# cf_fallback: uses user mean for all items
print(f"\nevaluating cf_fallback on cold items")
print("  (baseline: user mean or global mean)")

reg_metrics_cf = evaluate_regression(cf_cold.predict_fallback, test_cold)
print(f"  Regression: MAE={reg_metrics_cf['MAE']:.4f}, RMSE={reg_metrics_cf['RMSE']:.4f}")

print(f"  Computing ranking metrics...")
rank_metrics_cf = evaluate_ranking(
    cf_cold.predict_fallback, train_cold, test_cold,
    cold_candidate_pool, K=K, N_neg=N_NEG, max_users=MAX_USERS_RANK,
    relevance_threshold=RELEVANCE_THRESHOLD
)
print(f"  Ranking: Precision@{K}={rank_metrics_cf[f'Precision@{K}']:.4f}, "
      f"Recall@{K}={rank_metrics_cf[f'Recall@{K}']:.4f}, "
      f"NDCG@{K}={rank_metrics_cf[f'NDCG@{K}']:.4f}")

cold_results.append({
    'Model': 'CF_fallback',
    'Regime': 'Cold-Item',
    **reg_metrics_cf,
    **{k: v for k, v in rank_metrics_cf.items() if k != 'n_users_evaluated'}
})

# CBF: Ranking-only model (similarity scores)
print(f"\n--- Evaluating CBF on cold items")
print("  (Ranking-only: using similarity scores)")

print(f"  Computing ranking metrics...")
rank_metrics_cbf = evaluate_ranking(
    cbf_cold.predict, train_cold, test_cold,
    cold_candidate_pool, K=K, N_neg=N_NEG, max_users=MAX_USERS_RANK,
    relevance_threshold=RELEVANCE_THRESHOLD
)
print(f"  Ranking: Precision@{K}={rank_metrics_cbf[f'Precision@{K}']:.4f}, "
      f"Recall@{K}={rank_metrics_cbf[f'Recall@{K}']:.4f}, "
      f"NDCG@{K}={rank_metrics_cbf[f'NDCG@{K}']:.4f}")
print(f"  Regression: N/A (CBF is a ranking model, not a rating predictor)")

cold_results.append({
    'Model': 'CBF',
    'Regime': 'Cold-Item',
    'MAE': float('nan'),
    'RMSE': float('nan'),
    **{k: v for k, v in rank_metrics_cbf.items() if k != 'n_users_evaluated'}
})

# Hybrid: Uses CBF (similarity scores) for cold items
print(f"\n--- Evaluating Hybrid on cold items")
print("  (switching strategy: CBF similarity scores for cold items)")

print(f"  Computing ranking metrics...")
rank_metrics_hybrid = evaluate_ranking(
    cbf_cold.predict, train_cold, test_cold,
    cold_candidate_pool, K=K, N_neg=N_NEG, max_users=MAX_USERS_RANK,
    relevance_threshold=RELEVANCE_THRESHOLD
)
print(f"  Ranking: Precision@{K}={rank_metrics_hybrid[f'Precision@{K}']:.4f}, "
      f"Recall@{K}={rank_metrics_hybrid[f'Recall@{K}']:.4f}, "
      f"NDCG@{K}={rank_metrics_hybrid[f'NDCG@{K}']:.4f}")
print(f"  Regression: N/A (Hybrid uses CBF similarity for cold items)")

cold_results.append({
    'Model': 'Hybrid',
    'Regime': 'Cold-Item',
    'MAE': float('nan'),
    'RMSE': float('nan'),
    **{k: v for k, v in rank_metrics_hybrid.items() if k != 'n_users_evaluated'}
})

# create results dataframe
cold_results_df = pd.DataFrame(cold_results)
cold_results_df.to_csv(TABLES_DIR / 'cold_item_metrics.csv', index=False)

print(f"\n{'='*60}")
print("COLD ITEM REGIME RESULTS")
print(f"{'='*60}")
print(cold_results_df.to_string(index=False))
print(f"\nResults saved to {TABLES_DIR / 'cold_item_metrics.csv'}")


### 7.3 Few-shot User Regime Experiments


In [ ]:
# few-shot user experiments

print("\nfew-shot user regime experiments")
print("note: using consistent user set across all k values for fair comparison")

# use same users across all k values
# identify users who qualify for all k values (>k_max ratings)
k_max = max(fewshot_splits.keys())
user_rating_counts = ratings.groupby('userId').size()
common_users = set(user_rating_counts[user_rating_counts > k_max].index)

print(f"\nusing users with >{k_max} ratings for all k values")
print(f"common users across k=1,3,5: {len(common_users):,}")

# filter each split to only include common users
fewshot_splits_filtered = {}
for k, (train_k, test_k) in fewshot_splits.items():
    train_k_filtered = train_k[train_k['userId'].isin(common_users)].copy()
    test_k_filtered = test_k[test_k['userId'].isin(common_users)].copy()
    fewshot_splits_filtered[k] = (train_k_filtered, test_k_filtered)
    print(f"k={k}: {len(train_k_filtered):,} train, {len(test_k_filtered):,} test "
          f"({test_k_filtered['userId'].nunique():,} users)")

fewshot_results = []

for k, (train_k, test_k) in fewshot_splits_filtered.items():
    print(f"\n{'='*60}")
    print(f"FEW-SHOT k={k}")
    print(f"{'='*60}")
    print(f"Train size: {len(train_k):,}, Test size: {len(test_k):,}")
    
    # train CF model
    cf_k_cache_path = CACHE_DIR / f'cf_model_fewshot_k{k}.pkl'
    
    if cf_k_cache_path.exists():
        print(f"loading cached CF model (k={k})...")
        cf_k = CFModel.load(cf_k_cache_path)
    else:
        cf_k = CFModel(n_factors=50, n_epochs=15, random_state=SEED)  # Smaller for few-shot
        cf_k.fit(train_k, verbose=True)
        cf_k.save(cf_k_cache_path)
    
    # train CBF model (or load cached user profiles if available)
    cbf_k_cache_path = CACHE_DIR / f'cbf_profiles_fewshot_k{k}.pkl'
    
    if cbf_k_cache_path.exists():
        print(f"loading cached CBF model (k={k})...")
        cbf_k = CBFModel.load(cbf_k_cache_path)
    else:
        print(f"training new CBF model (k={k})...")
        cbf_k = CBFModel(use_rating_weights=True)
        cbf_k.fit(train_k, best_embeddings, best_movieId_to_idx, verbose=True)
        cbf_k.save(cbf_k_cache_path)
    
    # create switching hybrid
    hybrid_k = HybridModel(cf_k, cbf_k)
    
    # candidate pool: items in training set
    fewshot_candidate_pool = set(train_k['movieId'].unique())
    print(f"Candidate pool size: {len(fewshot_candidate_pool):,}")
    
    # sample test set for efficiency (100k ratings max)
    # use consistent seed offset across all k values for fair comparison
    if len(test_k) > 100000:
        print(f"  Sampling 100k test ratings from {len(test_k):,} for efficiency")
        test_k_sample = test_k.sample(n=100000, random_state=SEED + 100)  # Fixed seed
    else:
        test_k_sample = test_k
    
    # IMPORTANT: For regression evaluation, filter to only items in training set
    # this ensures fair comparison between CF and Hybrid (both evaluate on same items)
    # without this, Hybrid uses CBF similarity (0-1) for items not in train_k,
    # which produces terrible MAE/RMSE when compared against true ratings (1-5)
    test_k_regression = test_k_sample[test_k_sample['movieId'].isin(fewshot_candidate_pool)]
    print(f"  Regression test set: {len(test_k_regression):,} ratings (filtered to warm items only)")
    
    # evaluate CF
    print(f"\n--- Evaluating CF (k={k})")
    reg_metrics_cf = evaluate_regression(cf_k.predict, test_k_regression)
    print(f"  Regression: MAE={reg_metrics_cf['MAE']:.4f}, RMSE={reg_metrics_cf['RMSE']:.4f}")
    
    print(f"  Computing ranking metrics...")
    rank_metrics_cf = evaluate_ranking(
        cf_k.predict, train_k, test_k_sample,
        fewshot_candidate_pool, K=K, N_neg=N_NEG, max_users=2000,
        relevance_threshold=RELEVANCE_THRESHOLD
    )
    print(f"  Ranking: Precision@{K}={rank_metrics_cf[f'Precision@{K}']:.4f}, "
          f"Recall@{K}={rank_metrics_cf[f'Recall@{K}']:.4f}, "
          f"NDCG@{K}={rank_metrics_cf[f'NDCG@{K}']:.4f}")
    
    fewshot_results.append({
        'Model': 'CF',
        'Regime': f'Few-shot k={k}',
        'k': k,
        **reg_metrics_cf,
        **{key: val for key, val in rank_metrics_cf.items() if key != 'n_users_evaluated'}
    })
    
    # evaluate CBF (ranking-only)
    print(f"\n--- Evaluating CBF (k={k})")
    print(f"  (Ranking-only: using similarity scores)")
    
    print(f"  Computing ranking metrics...")
    rank_metrics_cbf = evaluate_ranking(
        cbf_k.predict, train_k, test_k_sample,
        fewshot_candidate_pool, K=K, N_neg=N_NEG, max_users=2000,
        relevance_threshold=RELEVANCE_THRESHOLD
    )
    print(f"  Ranking: Precision@{K}={rank_metrics_cbf[f'Precision@{K}']:.4f}, "
          f"Recall@{K}={rank_metrics_cbf[f'Recall@{K}']:.4f}, "
          f"NDCG@{K}={rank_metrics_cbf[f'NDCG@{K}']:.4f}")
    print(f"  Regression: N/A (CBF is a ranking model)")
    
    fewshot_results.append({
        'Model': 'CBF',
        'Regime': f'Few-shot k={k}',
        'k': k,
        'MAE': float('nan'),
        'RMSE': float('nan'),
        **{key: val for key, val in rank_metrics_cbf.items() if key != 'n_users_evaluated'}
    })
    
    # evaluate Hybrid
    print(f"\n--- Evaluating Hybrid (k={k})")
    reg_metrics_hybrid = evaluate_regression(hybrid_k.predict, test_k_regression)
    print(f"  Regression: MAE={reg_metrics_hybrid['MAE']:.4f}, RMSE={reg_metrics_hybrid['RMSE']:.4f}")
    
    print(f"  Computing ranking metrics...")
    rank_metrics_hybrid = evaluate_ranking(
        hybrid_k.predict, train_k, test_k_sample,
        fewshot_candidate_pool, K=K, N_neg=N_NEG, max_users=2000,
        relevance_threshold=RELEVANCE_THRESHOLD
    )
    print(f"  Ranking: Precision@{K}={rank_metrics_hybrid[f'Precision@{K}']:.4f}, "
          f"Recall@{K}={rank_metrics_hybrid[f'Recall@{K}']:.4f}, "
          f"NDCG@{K}={rank_metrics_hybrid[f'NDCG@{K}']:.4f}")
    
    fewshot_results.append({
        'Model': 'Hybrid',
        'Regime': f'Few-shot k={k}',
        'k': k,
        **reg_metrics_hybrid,
        **{key: val for key, val in rank_metrics_hybrid.items() if key != 'n_users_evaluated'}
    })

# create results dataframe
fewshot_results_df = pd.DataFrame(fewshot_results)
fewshot_results_df.to_csv(TABLES_DIR / 'fewshot_metrics.csv', index=False)

print(f"\n{'='*60}")
print("FEW-SHOT REGIME RESULTS")
print(f"{'='*60}")
print(fewshot_results_df.to_string(index=False))
print(f"\nResults saved to {TABLES_DIR / 'fewshot_metrics.csv'}")

# EXPECTED BEHAVIOR (with consistent user set):
# - k=1 → k=3 → k=5: Performance should INCREASE monotonically
# - More training data (higher k) → better user profiles → better predictions
# - If k=5 < k=3: likely due to overfitting or noise (check sample sizes)
print(f"\n{'='*60}")
print("INTERPRETATION (Consistent User Set):")
print(f"{'='*60}")
print("With same users across all k values:")
print("- k=1: Limited signal (1 rating/user) → Lower performance")
print("- k=3: More signal (3 ratings/user) → Better performance")  
print("- k=5: Most signal (5 ratings/user) → Best performance (expected)")
print("Hybrid ≈ CF because most test items are warm (in training set)")


## 8. Plots and Visualizations


In [ ]:
# 8.1 WARM PERFORMANCE VS ITEM POPULARITY

print()
print("PLOT 8.1: Warm Performance vs Item Popularity")
print()

# define models dictionary for plotting
models_warm = {
    'CF': cf_warm.predict,
    'CBF': cbf_warm.predict,
    'Hybrid': hybrid_warm.predict
}

# compute item popularity in training set
item_popularity = train_warm.groupby('movieId').size().to_dict()

# define popularity buckets
def get_popularity_bucket(pop):
    if pop <= 2:
        return '[1-2]'
    elif pop <= 10:
        return '[3-10]'
    elif pop <= 50:
        return '[11-50]'
    elif pop <= 200:
        return '[51-200]'
    else:
        return '[200+]'

# add bucket to test_warm
test_warm_with_pop = test_warm.copy()
test_warm_with_pop['popularity'] = test_warm_with_pop['movieId'].map(
    lambda x: item_popularity.get(x, 0)
)
test_warm_with_pop['bucket'] = test_warm_with_pop['popularity'].apply(get_popularity_bucket)

# order buckets
bucket_order = ['[1-2]', '[3-10]', '[11-50]', '[51-200]', '[200+]']

# compute NDCG@10 for each bucket
bucket_results = []

for bucket in bucket_order:
    bucket_test = test_warm_with_pop[test_warm_with_pop['bucket'] == bucket]
    if len(bucket_test) < 100:  # Skip if too few samples
        continue
    
    bucket_candidate_pool = warm_candidate_pool
    bucket_users = bucket_test['userId'].unique()
    
    # sample users for efficiency
    if len(bucket_users) > 1000:
        bucket_users = np.random.choice(bucket_users, size=1000, replace=False)
    
    for model_name, predict_fn in models_warm.items():
        # simple evaluation on bucket subset
        bucket_test_sample = bucket_test[bucket_test['userId'].isin(bucket_users)]
        
        rank_metrics = evaluate_ranking(
            predict_fn, train_warm, bucket_test_sample,
            bucket_candidate_pool, K=K, N_neg=N_NEG, max_users=1000,
            relevance_threshold=RELEVANCE_THRESHOLD
        )
        
        bucket_results.append({
            'Bucket': bucket,
            'Model': model_name,
            f'NDCG@{K}': rank_metrics[f'NDCG@{K}'],
            f'Recall@{K}': rank_metrics[f'Recall@{K}']
        })

bucket_results_df = pd.DataFrame(bucket_results)


In [ ]:
# create plot
fig, ax = plt.subplots(figsize=(10, 6))

for model_name in ['CF', 'CBF']:
    model_data = bucket_results_df[bucket_results_df['Model'] == model_name]
    if len(model_data) > 0:
        ax.plot(model_data['Bucket'], model_data[f'NDCG@{K}'], 
                marker='o', linewidth=2, markersize=8, label=model_name)

ax.set_xlabel('Item Popularity Bucket', fontsize=12)
ax.set_ylabel(f'NDCG@{K}', fontsize=12)
ax.set_title('Warm Regime: Performance vs Item Popularity', fontsize=14)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'warm_popularity_plot.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nPlot saved to {FIGURES_DIR / 'warm_popularity_plot.png'}")

In [ ]:
# 8.2 REGIME COMPARISON BAR CHARTS (SEPARATE FIGURES)

print()
print("PLOT 8.2: Regime Comparison")
print()

# combine all results
all_results = pd.concat([
    warm_results_df,
    cold_results_df,
    fewshot_results_df[fewshot_results_df['k'].isin([1, 3, 5])]
], ignore_index=True)

# create a simplified regime label
def simplify_regime(row):
    if 'Warm' in row['Regime']:
        return 'Warm'
    elif 'Cold' in row['Regime']:
        return 'Cold-Item'
    else:
        return row['Regime']

all_results['Regime_Simple'] = all_results.apply(simplify_regime, axis=1)

# prepare data for plotting
regimes_order = ['Warm', 'Cold-Item', 'Few-shot k=1', 'Few-shot k=3', 'Few-shot k=5']
models_order = ['CF', 'CBF', 'Hybrid', 'CF_fallback']
colors = {'CF': '#2ecc71', 'CBF': '#3498db', 'Hybrid': '#e74c3c', 'CF_fallback': '#95a5a6'}

x = np.arange(len(regimes_order))
width = 0.25

# helper function to get values for a metric
def get_metric_values(metric_col, model):
    values = []
    for regime in regimes_order:
        if regime == 'Cold-Item' and model == 'CF':
            # use CF_fallback for cold regime
            model_data = all_results[(all_results['Regime'] == regime) & (all_results['Model'] == 'CF_fallback')]
        else:
            model_data = all_results[(all_results['Regime'] == regime) & (all_results['Model'] == model)]
        
        if len(model_data) > 0:
            values.append(model_data[metric_col].values[0])
        else:
            values.append(0)
    return values

# FIGURE 1: Recall@K
fig1, ax1 = plt.subplots(figsize=(10, 6))
recall_col = f'Recall@{K}'

multiplier = 0
for model in ['CF', 'CBF', 'Hybrid']:
    values = get_metric_values(recall_col, model)
    offset = width * multiplier
    ax1.bar(x + offset, values, width, label=model, color=colors.get(model, '#333'))
    multiplier += 1

ax1.set_xlabel('Regime', fontsize=13, fontweight='bold')
ax1.set_ylabel(recall_col, fontsize=13, fontweight='bold')
ax1.set_title(f'{recall_col} Comparison Across Regimes', fontsize=15, fontweight='bold')
ax1.set_xticks(x + width)
ax1.set_xticklabels(regimes_order, rotation=15, ha='right', fontsize=11)
ax1.legend(fontsize=11, loc='upper right')
ax1.grid(True, alpha=0.3, axis='y')
ax1.set_ylim(0, max([max(get_metric_values(recall_col, m)) for m in ['CF', 'CBF', 'Hybrid']]) * 1.15)

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'regime_comparison_recall.png', dpi=150, bbox_inches='tight')
plt.show()
print(f"Saved: {FIGURES_DIR / 'regime_comparison_recall.png'}")

# FIGURE 2: NDCG@K
fig2, ax2 = plt.subplots(figsize=(10, 6))
ndcg_col = f'NDCG@{K}'

multiplier = 0
for model in ['CF', 'CBF', 'Hybrid']:
    values = get_metric_values(ndcg_col, model)
    offset = width * multiplier
    ax2.bar(x + offset, values, width, label=model, color=colors.get(model, '#333'))
    multiplier += 1

ax2.set_xlabel('Regime', fontsize=13, fontweight='bold')
ax2.set_ylabel(ndcg_col, fontsize=13, fontweight='bold')
ax2.set_title(f'{ndcg_col} Comparison Across Regimes', fontsize=15, fontweight='bold')
ax2.set_xticks(x + width)
ax2.set_xticklabels(regimes_order, rotation=15, ha='right', fontsize=11)
ax2.legend(fontsize=11, loc='upper right')
ax2.grid(True, alpha=0.3, axis='y')
ax2.set_ylim(0, max([max(get_metric_values(ndcg_col, m)) for m in ['CF', 'CBF', 'Hybrid']]) * 1.15)

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'regime_comparison_ndcg.png', dpi=150, bbox_inches='tight')
plt.show()
print(f"Saved: {FIGURES_DIR / 'regime_comparison_ndcg.png'}")

# FIGURE 3: Precision@K
fig3, ax3 = plt.subplots(figsize=(10, 6))
precision_col = f'Precision@{K}'

multiplier = 0
for model in ['CF', 'CBF', 'Hybrid']:
    values = get_metric_values(precision_col, model)
    offset = width * multiplier
    ax3.bar(x + offset, values, width, label=model, color=colors.get(model, '#333'))
    multiplier += 1

ax3.set_xlabel('Regime', fontsize=13, fontweight='bold')
ax3.set_ylabel(precision_col, fontsize=13, fontweight='bold')
ax3.set_title(f'{precision_col} Comparison Across Regimes', fontsize=15, fontweight='bold')
ax3.set_xticks(x + width)
ax3.set_xticklabels(regimes_order, rotation=15, ha='right', fontsize=11)
ax3.legend(fontsize=11, loc='upper right')
ax3.grid(True, alpha=0.3, axis='y')
ax3.set_ylim(0, .5)  # Precision is between 0 and 1

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'regime_comparison_precision.png', dpi=150, bbox_inches='tight')
plt.show()
print(f"Saved: {FIGURES_DIR / 'regime_comparison_precision.png'}")

print(f"\n{'='*60}")
print("All regime comparison plots saved!")
print(f"{'='*60}")


## 9. Final Summary and Key Findings


In [ ]:
# 9. FINAL SUMMARY

print()
print("EXPERIMENT SUMMARY")
print()

# list all saved files
print("\n📁 SAVED FILES:")
print("-" * 50)

tables_files = list(TABLES_DIR.glob('*.csv'))
figures_files = list(FIGURES_DIR.glob('*.png'))

print("\n📊 Tables:")
for f in sorted(tables_files):
    print(f"  - {f}")

print("\n📈 Figures:")
for f in sorted(figures_files):
    print(f"  - {f}")

print("\n💾 Cache files (for faster re-runs):")
cache_files = list(CACHE_DIR.glob('*'))
for f in sorted(cache_files)[:10]:  # Show first 10
    print(f"  - {f}")
if len(cache_files) > 10:
    print(f"  ... and {len(cache_files) - 10} more")

# key findings
print("\n" + "="*70)
print("KEY FINDINGS")
print()

print(f"\n1️⃣  BEST EMBEDDING MODEL: {best_embedding_model}")
print(f"    Selected based on CBF performance on warm validation set")

print(f"\n2️⃣  HYBRID STRATEGY: switching (No Blending)")
print(f"    Warm items → CF ratings (best for rating prediction)")
print(f"    Cold items → CBF similarity (only model that works)")
print(f"    No alpha parameter - simple gating based on item warmth")

# find winners for each regime
print("\n3️⃣  WINNERS BY REGIME (based on NDCG@10):")

# warm winner
warm_winner = warm_results_df.loc[warm_results_df[f'NDCG@{K}'].idxmax(), 'Model']
warm_best_ndcg = warm_results_df[f'NDCG@{K}'].max()
print(f"    - Warm: {warm_winner} (NDCG@{K}={warm_best_ndcg:.4f})")

# cold winner
cold_winner = cold_results_df.loc[cold_results_df[f'NDCG@{K}'].idxmax(), 'Model']
cold_best_ndcg = cold_results_df[f'NDCG@{K}'].max()
print(f"    - Cold-Item: {cold_winner} (NDCG@{K}={cold_best_ndcg:.4f})")

# few-shot winners
for k in [1, 3, 5]:
    fs_df = fewshot_results_df[fewshot_results_df['k'] == k]
    if len(fs_df) > 0:
        fs_winner = fs_df.loc[fs_df[f'NDCG@{K}'].idxmax(), 'Model']
        fs_best_ndcg = fs_df[f'NDCG@{K}'].max()
        print(f"    - Few-shot k={k}: {fs_winner} (NDCG@{K}={fs_best_ndcg:.4f})")

# hybrid robustness
print("\n4️⃣  HYBRID MODEL ROBUSTNESS:")
print("    The gated hybrid approach provides:")
print("    - Competitive performance on warm items (benefiting from CF)")
print("    - Graceful degradation on cold items (fallback to CBF)")
print("    - Consistent results across few-shot scenarios")

# performance patterns
print("\n5️⃣  PERFORMANCE PATTERNS:")
print("    - CF excels on warm items with rich interaction history")
print("    - CBF provides crucial coverage for cold items")
print("    - Performance degrades from warm → cold → few-shot")
print("    - Hybrid balances both approaches effectively")

print("\n" + "="*70)
print("EXPERIMENT COMPLETED SUCCESSFULLY")
print()
print(f"\nTotal tables saved: {len(tables_files)}")
print(f"Total figures saved: {len(figures_files)}")
print(f"Total cache files: {len(cache_files)}")


In [ ]:
# COMBINED RESULTS TABLE (Paper-Ready)

print()
print("COMBINED RESULTS TABLE (Paper-Ready)")
print()

# combine all results into one DataFrame
combined_results = pd.concat([
    warm_results_df.assign(Regime_Order=1),
    cold_results_df.assign(Regime_Order=2),
    fewshot_results_df.assign(Regime_Order=3)
], ignore_index=True)

# select and order columns
display_cols = ['Regime', 'Model', 'MAE', 'RMSE', f'Recall@{K}', f'NDCG@{K}', f'Precision@{K}']
combined_display = combined_results[display_cols].copy()

# format numeric columns
for col in ['MAE', 'RMSE', f'Recall@{K}', f'NDCG@{K}', f'Precision@{K}']:
    combined_display[col] = combined_display[col].apply(lambda x: f'{x:.4f}')

print("\n" + combined_display.to_string(index=False))

# save combined results
combined_results[display_cols].to_csv(TABLES_DIR / 'combined_results.csv', index=False)
print(f"\nCombined results saved to {TABLES_DIR / 'combined_results.csv'}")


## 9. Qualitative Analysis: Example Users and Recommendations

Show concrete examples of what each model recommends for real users across different regimes.


In [ ]:
# qualitative analysis helper

movies_with_titles = pd.read_csv(DATA_DIR / 'movies_enriched_big.csv', 
                                  usecols=['movieId', 'title'])
print(f"loaded {len(movies_with_titles):,} movie titles")

# shows recommendations for a user and compares with actual test ratings
def show_user_recommendations(user_id, train_df, test_df, models_dict, movies_df, 
                               regime_name, top_k=10, show_train_sample=5):
    # get user's training and test items
    user_train = train_df[train_df['userId'] == user_id].sort_values('rating', ascending=False)
    user_test = test_df[test_df['userId'] == user_id].sort_values('rating', ascending=False)
    
    if len(user_test) == 0:
        print(f"user {user_id} has no test data, skipping")
        return
    
    print(f"\nuser {user_id} - {regime_name} regime")
    print("-" * 80)
    
    # training history sample
    print(f"\ntraining history ({len(user_train)} items, top {show_train_sample} by rating):")
    for idx, row in user_train.head(show_train_sample).iterrows():
        movie_info = movies_with_titles[movies_with_titles['movieId'] == row['movieId']]
        if len(movie_info) > 0:
            title = movie_info.iloc[0]['title']
            print(f"  {row['rating']:.1f} - {title}")
    
    # test ground truth (highly rated)
    print(f"\ntest ground truth ({len(user_test)} items, showing ratings >= 4.0):")

    high_rated_test = user_test[user_test['rating'] >= 4.0]
    if len(high_rated_test) > 0:
        for idx, row in high_rated_test.head(10).iterrows():
            movie_info = movies_with_titles[movies_with_titles['movieId'] == row['movieId']]
            if len(movie_info) > 0:
                title = movie_info.iloc[0]['title']
                print(f"  {row['rating']:.1f} - {title}")
    else:
        print("  (no ratings >= 4.0)")
    
    # candidate items are test items not in training
    user_train_items = set(user_train['movieId'].values)
    user_test_items = set(user_test['movieId'].values)
    candidate_items = list(user_test_items - user_train_items)
    
    if len(candidate_items) == 0:
        print("\nno new items to recommend")
        return
    
    # prepare pairs for scoring
    pairs_df = pd.DataFrame({
        'userId': [user_id] * len(candidate_items),
        'movieId': candidate_items
    })
    
    # get actual ratings
    actual_ratings = test_df[
        (test_df['userId'] == user_id) & 
        (test_df['movieId'].isin(candidate_items))
    ].set_index('movieId')['rating'].to_dict()
    
    # show recommendations from each model
    for model_name, predict_fn in models_dict.items():
        print(f"\n{model_name.upper()} recommendations (top {top_k}):")
        
        scores = predict_fn(pairs_df)
        ranked_indices = np.argsort(-scores)
        
        # display top-k
        for i, idx in enumerate(ranked_indices[:top_k]):
            movie_id = candidate_items[idx]
            pred_score = scores[idx]
            actual_rating = actual_ratings.get(movie_id, None)
            
            movie_info = movies_with_titles[movies_with_titles['movieId'] == movie_id]
            if len(movie_info) > 0:
                title = movie_info.iloc[0]['title']
                
                if actual_rating is not None:
                    match_symbol = "+" if actual_rating >= 4.0 else "-"
                    print(f"  {i+1:2d}. {title[:60]:60s} | pred: {pred_score:.2f} | actual: {actual_rating:.1f} {match_symbol}")
                else:
                    print(f"  {i+1:2d}. {title[:60]:60s} | pred: {pred_score:.2f} | actual: n/a")
        
        # compute precision for this user
        top_k_items = [candidate_items[idx] for idx in ranked_indices[:top_k]]
        hits = [item for item in top_k_items if actual_ratings.get(item, 0) >= 4.0]
        precision = len(hits) / top_k if top_k > 0 else 0
        print(f"\n  precision@{top_k} (>=4.0): {len(hits)}/{top_k} = {precision:.2%}")
    
    print()


print("helper function defined")


### 9.2 Warm Regime Examples

Show examples of recommendations for users in the warm regime.


In [ ]:
# 9.2 WARM REGIME EXAMPLES

print("\n" + "="*100)
print("WARM REGIME: Qualitative Examples")
print()

# select a few interesting users (with diverse preferences)
# pick users with reasonable amount of test data
test_user_counts = test_warm.groupby('userId').size()
candidate_users = test_user_counts[test_user_counts >= 20].index.tolist()

# sample 2 users for detailed analysis
np.random.seed(SEED)
example_users_warm = np.random.choice(candidate_users, size=2, replace=False)

# models to compare
# note: CBF shows similarity scores (not ratings)
models_warm_qual = {
    'CF (ratings)': cf_warm.predict,
    'CBF (similarity)': cbf_warm.predict,  # Returns similarity scores
    'Hybrid (ratings)': hybrid_warm.predict
}

print("note: CBF shows SIMILARITY SCORES (not ratings)")
print("Higher similarity = more relevant. Scores typically range from 0 to 1.")
print()

# show examples
for user_id in example_users_warm:
    show_user_recommendations(
        user_id=user_id,
        train_df=train_warm,
        test_df=test_warm,
        models_dict=models_warm_qual,
        movies_df=movies_with_titles,
        regime_name="Warm",
        top_k=10,
        show_train_sample=5
    )


### 9.3 Cold-Start Item Regime Examples

Show examples of recommendations for cold items (items that didn't exist in training).


In [ ]:
# 9.3 COLD-START ITEM REGIME EXAMPLES

print("\n" + "="*100)
print("COLD-ITEM REGIME: Qualitative Examples")
print()

# select users who rated cold items AND have training data (for CBF profile)
train_cold_users = set(train_cold['userId'].unique())
test_cold_user_counts = test_cold.groupby('userId').size()

# IMPORTANT: User must be in train_cold to have a CBF profile
# select users with MORE test items (> =20) for better qualitative analysis
candidate_users_cold = [
    uid for uid, count in test_cold_user_counts.items() 
    if count >= 20 and uid in train_cold_users  # Increased from 10 to 20
]

print(f"Candidate users (≥20 test items, in both train & test): {len(candidate_users_cold)}")

if len(candidate_users_cold) < 2:
    print("warning: Not enough users with ≥20 test items, lowering threshold to ≥15")
    candidate_users_cold = [
        uid for uid, count in test_cold_user_counts.items() 
        if count >= 15 and uid in train_cold_users
    ]
    print(f"Candidate users (≥15 test items): {len(candidate_users_cold)}")

# sample 2 users
np.random.seed(SEED + 1)
example_users_cold = np.random.choice(candidate_users_cold, size=2, replace=False)

# print selected user test counts
for uid in example_users_cold:
    n_test = len(test_cold[test_cold['userId'] == uid])
    print(f"  User {uid}: {n_test} cold items in test set")

# models to compare (CF_fallback instead of regular CF)
# for cold-start qualitative analysis:
# - CF_fallback: uses user mean (for reference)
# - CBF: uses SIMILARITY SCORES (not ratings)
# - Hybrid: uses CBF similarity scores for cold items

models_cold_qual = {
    'CF_fallback (ratings)': cf_cold.predict_fallback,
    'CBF (similarity)': cbf_cold.predict,  # CBF.predict returns similarity scores
    'Hybrid (similarity)': cbf_cold.predict  # Same as CBF for cold items
}

print("note: CBF and Hybrid show SIMILARITY SCORES (not ratings) for cold-start")
print("Higher similarity = more relevant. Scores typically range from 0 to 1.")
print()

# show examples
for user_id in example_users_cold:
    show_user_recommendations(
        user_id=user_id,
        train_df=train_cold,
        test_df=test_cold,
        models_dict=models_cold_qual,
        movies_df=movies_with_titles,
        regime_name="Cold-Item",
        top_k=10,
        show_train_sample=5
    )


### 9.4 Few-Shot User Regime Examples

Show examples of recommendations for users with limited training data (k=1, 3, 5).


In [ ]:
# 9.4 FEW-SHOT USER REGIME EXAMPLES

# show examples for k=1, k=3, and k=5
for k in [1, 3, 5]:
    print(f"# FEW-SHOT k={k}: Users with only {k} training rating(s)")
    
    train_k, test_k = fewshot_splits[k]
    
    # select users with reasonable test data
    test_fewshot_counts = test_k.groupby('userId').size()
    candidate_users_fewshot = test_fewshot_counts[test_fewshot_counts >= 20].index.tolist()
    
    # sample 1 user per k (to keep output manageable)
    np.random.seed(SEED + k)
    example_user = np.random.choice(candidate_users_fewshot, size=1)[0]
    
    # need to re-train models for this k (or use cached if available)
    # for qualitative analysis, we'll use the last trained models from the few-shot experiments
    # note: This assumes the few-shot experiment loop has completed
    
    print(f"note: Using models trained on few-shot k={k} data from earlier experiments.")
    print(f"training set size: {len(train_k):,} ratings")
    print(f"Test set size: {len(test_k):,} ratings\n")
    
    # for demonstration, we'll train quick models here
    # CF model
    cf_k = CFModel(n_factors=50, n_epochs=15, random_state=SEED)
    cf_k.fit(train_k, verbose=False)
    
    # CBF model (use cached profile if available)
    cbf_k = CBFModel(use_rating_weights=True)
    cbf_k.fit(train_k, best_embeddings, best_movieId_to_idx, verbose=False)
    
    # switching hybrid
    hybrid_k = HybridModel(cf_k, cbf_k)
    
    # models dict
    # note: CBF shows similarity scores (not ratings)
    models_fewshot_qual = {
        'CF (ratings)': cf_k.predict,
        'CBF (similarity)': cbf_k.predict,  # Returns similarity scores
        'Hybrid (ratings)': hybrid_k.predict
    }
    
    if k == 1:  # Only print once
        print("note: CBF shows SIMILARITY SCORES (not ratings)")
        print("Higher similarity = more relevant. Scores typically range from 0 to 1.")
        print()
    
    # show recommendation
    show_user_recommendations(
        user_id=example_user,
        train_df=train_k,
        test_df=test_k,
        models_dict=models_fewshot_qual,
        movies_df=movies_with_titles,
        regime_name=f"Few-shot k={k}",
        top_k=10,
        show_train_sample=k  # Show all k training examples
    )
